In [2]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.feature_extraction.text import CountVectorizer
import pickle as pkl
import networkx as nx
from networkx.readwrite import json_graph
import json

In [46]:
data = pd.read_excel('add_patent.xlsx', sheet_name='분석대상', index_col=None, header=None)
data = data.fillna(' ')
data = data.loc[:, (data != ' ').any(axis=0)]
data = data.iloc[1:,1:-1]
data = data.reset_index(drop=True)
data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1979,d_5109,H04J,,,,,,,,,,,,,,
1,1980,d_5106,H04L,,,,,,,,,,,,,,
2,1981,d_5107,A01M,,,,,,,,,,,,,,
3,1981,d_5108,C22F,,,,,,,,,,,,,,
4,1983,d_5104,G01R,G01R,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104,2020,d_0417,G06F,H04L,,,,,,,,,,,,,
5105,2020,d_0418,G01S,B63G,,,,,,,,,,,,,
5106,2020,d_0488,H04N,H04L,H04N,H04N,,,,,,,,,,,
5107,2020,d_0536,G01S,G01S,,,,,,,,,,,,,


In [64]:
test = data[data[1].isin(['2019', '2020'])]
# test = data[data[1].isin(['2020'])]
test = test.reset_index(drop=True)
test = test.iloc[:,2:]; test.iloc[:2]

,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,G01R,G01R,G01R,,,,,,,,,,,,
1,B01J,B01J,B01J,B01J,B01J,C06D,,,,,,,,,


In [65]:
train = data[data[1]!='2019']
train = train[train[1]!='2020']
train = train.reset_index(drop=True)

In [66]:
train

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1979,d_5109,H04J,,,,,,,,,,,,,,
1,1980,d_5106,H04L,,,,,,,,,,,,,,
2,1981,d_5107,A01M,,,,,,,,,,,,,,
3,1981,d_5108,C22F,,,,,,,,,,,,,,
4,1983,d_5104,G01R,G01R,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4614,2018,d_1381,B63G,H01F,,,,,,,,,,,,,
4615,2018,d_1409,B64C,,,,,,,,,,,,,,
4616,2018,d_1442,F02K,F01D,F02K,,,,,,,,,,,,
4617,2018,d_1462,H01Q,H01Q,H01Q,,,,,,,,,,,,


In [67]:
train = train.iloc[:,2:]; train

,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,H04J,,,,,,,,,,,,,,
1,H04L,,,,,,,,,,,,,,
2,A01M,,,,,,,,,,,,,,
3,C22F,,,,,,,,,,,,,,
4,G01R,G01R,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4614,B63G,H01F,,,,,,,,,,,,,
4615,B64C,,,,,,,,,,,,,,
4616,F02K,F01D,F02K,,,,,,,,,,,,
4617,H01Q,H01Q,H01Q,,,,,,,,,,,,


In [68]:
(len(train) + len(test)) == len(data)

True

In [69]:
train_merged = train[3].map(str)
for i in range(3, len(train.columns)+3):
    train_merged += ' ' + train[i].map(str)
train_merged

0             H04J H04J                            
1             H04L H04L                            
2             A01M A01M                            
3             C22F C22F                            
4          G01R G01R G01R                          
                           ...                     
4614       B63G B63G H01F                          
4615          B64C B64C                            
4616    F02K F02K F01D F02K                        
4617    H01Q H01Q H01Q H01Q                        
4618    H04L H04L H04B H04L                        
Name: 3, Length: 4619, dtype: object

In [70]:
train_idx = train_merged.index
cv = CountVectorizer()
cv.fit(train_merged)
results = cv.transform(train_merged)
train_ipc = cv.get_feature_names()
ipc_bow = pd.DataFrame(results.toarray(), columns=train_ipc, index=train_idx)
ipc_bow

,a01k,a01m,a01n,a41d,a42b,a47b,a61b,a61f,a61h,a61k,...,h04m,h04n,h04q,h04r,h04s,h04w,h05b,h05f,h05h,h05k
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4614,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4615,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4616,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4617,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
train_adj = np.matmul(ipc_bow.values.T, ipc_bow.values)
train_adj[train_adj != 0] = 1
np.fill_diagonal(train_adj, 0)
print(train_adj)
print(train_adj.shape)
print(np.count_nonzero(train_adj))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(322, 322)
2748


In [72]:
train_adj_idx = pd.DataFrame(train_adj, columns=train_ipc, index=train_ipc); train_adj_idx

,a01k,a01m,a01n,a41d,a42b,a47b,a61b,a61f,a61h,a61k,...,h04m,h04n,h04q,h04r,h04s,h04w,h05b,h05f,h05h,h05k
a01k,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
a01m,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a01n,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a41d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a42b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
h04w,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
h05b,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
h05f,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
h05h,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
# train_adj_idx.to_csv('add_ipc_train.csv')

In [74]:
test_merged = test[3].map(str)
for i in range(3, len(test.columns)+3):
    test_merged += ' ' + test[i].map(str)
test_merged

0            G01R G01R G01R G01R                        
1      B01J B01J B01J B01J B01J B01J C06D            ...
2                  H04K H04K                            
3         H01M H01M B01F B63G H01M                      
4               G01S G01S G01S                          
                             ...                        
485             G06F G06F H04L                          
486             G01S G01S B63G                          
487       H04N H04N H04L H04N H04N                      
488             G01S G01S G01S                          
489       G01S G01S G01S G02B G02B                      
Name: 3, Length: 490, dtype: object

#### TEST

In [75]:
test_idx = test.index
cv2 = CountVectorizer()
cv2.fit(test_merged)
results2 = cv2.transform(test_merged)
test_ipc = cv2.get_feature_names()
ipc_bow2 = pd.DataFrame(results2.toarray(), columns=test_ipc, index=test_idx)
ipc_bow2

,a41d,a61b,a61h,a61k,a61m,a61p,a62b,a62d,a63f,b01d,...,h04b,h04j,h04k,h04l,h04m,h04n,h04w,h05f,h05h,h05k
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
486,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
487,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,4,0,0,0,0
488,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
test_adj = np.matmul(ipc_bow2.values.T, ipc_bow2.values)
test_adj[test_adj != 0] = 1
np.fill_diagonal(test_adj, 0)
print(test_adj)
print(test_adj.shape)
print(np.count_nonzero(test_adj))

[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(153, 153)
814


In [77]:
test_adj_idx = pd.DataFrame(test_adj, columns=test_ipc, index=test_ipc); test_adj_idx.head()

,a41d,a61b,a61h,a61k,a61m,a61p,a62b,a62d,a63f,b01d,...,h04b,h04j,h04k,h04l,h04m,h04n,h04w,h05f,h05h,h05k
a41d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a61b,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a61h,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a61k,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a61m,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
G = nx.from_pandas_adjacency(test_adj_idx)

In [79]:
len(G.edges())

407

In [80]:
labeled_graph = train_adj_idx.copy()
np.count_nonzero(labeled_graph.values)

2748

In [81]:
for i, j in G.edges():
    if all(x in train_ipc for x in [i, j]) and (train_adj_idx.loc[i, j] == 0) and (train_adj_idx.loc[j, i] == 0):
        labeled_graph.loc[i, j] += 1
        labeled_graph.loc[j, i] += 1

In [82]:
np.count_nonzero(labeled_graph.values)

3102

In [83]:
labeled_graph

,a01k,a01m,a01n,a41d,a42b,a47b,a61b,a61f,a61h,a61k,...,h04m,h04n,h04q,h04r,h04s,h04w,h05b,h05f,h05h,h05k
a01k,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
a01m,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a01n,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a41d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a42b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
h04w,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
h05b,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
h05f,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
h05h,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
original_G = nx.from_pandas_adjacency(labeled_graph)
original_G_json = json_graph.node_link_data(original_G)

In [85]:
with open('original.graph', 'w') as f:
    json.dump(original_G_json, f)

In [86]:
train_G = nx.from_pandas_adjacency(train_adj_idx)
train_G_json = json_graph.node_link_data(train_G)

In [87]:
with open('train.graph', 'w') as f:
    json.dump(train_G_json, f)

In [88]:
test_G = nx.from_pandas_adjacency(test_adj_idx)
test_G_json = json_graph.node_link_data(test_G)

In [89]:
with open('test.graph', 'w') as f:
    json.dump(test_G_json, f)

#### Test load

In [57]:
with open('original.graph', 'r') as f:
    loaded = json.load(f)

In [58]:
test_load = json_graph.node_link_graph(loaded)

In [59]:
graphs = ['original', 'train', 'test']
obj = []
for graph in graphs:
    with open(graph +'.graph', 'r') as f:
        data = json.load(f)
    obj.append(data)
orig_g = json_graph.node_link_graph(obj[0])
train_g = json_graph.node_link_graph(obj[1])
test_g = json_graph.node_link_graph(obj[2])

In [60]:
print(len(orig_g.nodes()), len(orig_g.edges()))
print(len(train_g.nodes()), len(train_g.edges()))
print(len(test_g.nodes()), len(test_g.edges()))

322 1551
322 1374
153 407
